In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys
import boto3
import yaml

current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.insert(0, root_dir)

from scripts.common_functions import *

In [ ]:
access_key, access_secret = get_aws_creds()

In [3]:
# s3_client = boto3.client('s3')
# response = s3_client.list_objects_v2(Bucket='cricket-bucket-obi')
# for item in response.get('Contents', []):
#     print(item['Key'])